<a href="https://colab.research.google.com/github/margaridagomes/dataeng-basic-course/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: date - yyyy-mm-dd or yyyymmdd)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [ ]:
%pip install pyspark

In [ ]:
# Set up path in the "lake"
!mkdir -p /content/lake/bronze

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F
import requests

class ETLFlow:
    """
    Base ETL class that provides common extraction and loading functionality.
    """

    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def extract_from_api(self, url: str, schema: StructType = None):
        """
        Extract data from a REST API endpoint and load it into a Spark DataFrame, enforcing a schema using StructType.
        """
        response = requests.get(url)
        rdd = spark.sparkContext.parallelize(response.json())
        if schema:
            df = spark.read.schema(schema).json(rdd)
        else:
            df = spark.read.json(rdd)
        return df

    def load(self, df: DataFrame, format: str, path: str, partition_column: str = None, dynamic_partition_overwrite: bool = False, **kwargs) -> None:
        """
        Save the DataFrame to the specified path in the chosen format.
        If 'partition_column' is provided, partition the data accordingly.
        If 'dynamic_partition_overwrite' is True, only overwrite the partitions present in the DataFrame.
        """
        # Set Spark configuration for dynamic/static partition overwrite mode
        if dynamic_partition_overwrite:
            self.spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
        else:
            self.spark.conf.set("spark.sql.sources.partitionOverwriteMode", "static")

        # Write the DataFrame as a single parquet file, with optional partitioning
        if partition_column:
            df.coalesce(1).write.mode("overwrite").partitionBy(partition_column).format(format).save(path)
        else:
            df.coalesce(1).write.mode("overwrite").format(format).save(path)

class ETLTask(ETLFlow):
    """
    ETLTask class implements ingestion steps for vehicles, lines, and municipalities datasets.
    """

    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def ingestion_lines(self):
        """
        Extract 'lines' data from the API, enforce schema, and write as a single parquet file.
        """
        # Schema definition for lines
        lines_schema = StructType([StructField('color', StringType(), True),
                                   StructField('facilities', ArrayType(StringType(), True), True),
                                   StructField('id', StringType(), True),
                                   StructField('localities', ArrayType(StringType(), True), True),
                                   StructField('long_name', StringType(), True),
                                   StructField('municipalities', ArrayType(StringType(), True), True),
                                   StructField('patterns', ArrayType(StringType(), True), True),
                                   StructField('routes', ArrayType(StringType(), True), True),
                                   StructField('short_name', StringType(), True),
                                   StructField('text_color', StringType(), True)])

        # Ingestion
        df = self.extract_from_api(url="https://api.carrismetropolitana.pt/lines", schema=lines_schema)
        # Load
        self.load(df=df, format="parquet", path="/content/lake/bronze/lines")

    def ingestion_vehicles(self):
        """
        Extract 'vehicles' data from the API, enforce schema, add a partitioning 'date' column,
        and write as a partitioned parquet file with dynamic partition overwrite enabled.
        """
        # Schema definition for vehicles
        vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                                  StructField('block_id', StringType(), True),
                                  StructField('current_status', StringType(), True),
                                  StructField('id', StringType(), True),
                                  StructField('lat', FloatType(), True),
                                  StructField('line_id', StringType(), True),
                                  StructField('lon', FloatType(), True),
                                  StructField('pattern_id', StringType(), True),
                                  StructField('route_id', StringType(), True),
                                  StructField('schedule_relationship', StringType(), True),
                                  StructField('shift_id', StringType(), True),
                                  StructField('speed', FloatType(), True),
                                  StructField('stop_id', StringType(), True),
                                  StructField('timestamp', TimestampType(), True),
                                  StructField('trip_id', StringType(), True)])

        # Ingestion
        df = self.extract_from_api(url="https://api.carrismetropolitana.pt/vehicles", schema=vehicle_schema)

        # Create "date" column from "timestamp"
        df = df.withColumn("date", F.expr("date(timestamp)"))

        # Load
        self.load(df=df, format="parquet", path="/content/lake/bronze/vehicles", partition_column="date", dynamic_partition_overwrite=True)

    def ingestion_municipalities(self):
        """
        Extract 'municipalities' data from the API, enforce schema, and write as a single parquet file.
        """
        # Schema definition for municipalities
        municipalities_schema = StructType([StructField('district_id', StringType(), True),
                                            StructField('district_name', StringType(), True),
                                            StructField('id', StringType(), True),
                                            StructField('name', StringType(), True),
                                            StructField('prefix', StringType(), True),
                                            StructField('region_id', StringType(), True),
                                            StructField('region_name', StringType(), True)])

        # Ingestion
        df = self.extract_from_api(url="https://api.carrismetropolitana.pt/municipalities", schema=municipalities_schema)

        # Load
        self.load(df=df, format="parquet", path="/content/lake/bronze/municipalities")


if __name__ == '__main__':
    # Stop Spark if already running (prevents "Only one SparkContext" error)
    try:
        spark.stop()
    except:
        pass

    # Now initialize Spark
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.master('local').appName('ETL Program').getOrCreate()

    try:
        print("Starting ETL program")
        etl = ETLTask(spark)

        # Run all ingestion tasks
        print("Running Task - Ingestion Vehicles")
        etl.ingestion_vehicles()

        print("Running Task - Ingestion Lines")
        etl.ingestion_lines()

        print("Running Task - Ingestion Municipalities")
        etl.ingestion_municipalities()

        print("ETL program completed.")

        # Check results
        print("Preview - vehicles (bronze)")
        spark.read.parquet("/content/lake/bronze/vehicles").show(truncate=False)

        print("Preview - lines (bronze)")
        spark.read.parquet("/content/lake/bronze/lines").show(truncate=False)

        print("Preview - municipalities (bronze)")
        spark.read.parquet("/content/lake/bronze/municipalities").show(truncate=False)

    finally:
        # Clean up
        spark.stop()

+-------+------------------------------+--------------+--------+---------+-------+----------+----------+--------+---------------------+------------+---------+-------+-------------------+----------------------------------+----------+
|bearing|block_id                      |current_status|id      |lat      |line_id|lon       |pattern_id|route_id|schedule_relationship|shift_id    |speed    |stop_id|timestamp          |trip_id                           |date      |
+-------+------------------------------+--------------+--------+---------+-------+----------+----------+--------+---------------------+------------+---------+-------+-------------------+----------------------------------+----------+
|64     |20250705-64020039-112090000007|STOPPED_AT    |44|12083|38.525925|4426   |-8.9100485|4426_0_1  |4426_0  |SCHEDULED            |112090000007|0.0      |160264 |2025-07-05 15:02:04|4426_0_1|1300|1600_PYPHK          |2025-07-05|
|102    |20250705-64020292-121140000007|IN_TRANSIT_TO |44|12574|38.5